# Importing libraries and reading datasets

In [2]:
import pandas as pd
import os
import sweetviz as sv
import geopandas as gpd
from shapely import wkt
import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sns
import geodatasets
from geopy.geocoders import Nominatim
import warnings
import plotly.express as px
from geopy.geocoders import ArcGIS
import geopandas as gpd
import re
import openai
import os
import json
from icecream import ic
warnings.filterwarnings('ignore')

In [3]:
os.chdir('C:/Users/edwin/OneDrive/Escritorio/Personal files/Courses/Semester 2/Capstone Project/Databases')

In [4]:
df=pd.read_csv("dataframe_withlocations.csv").drop(columns=['Unnamed: 0'])
df.head()

,BusinessNo,Category,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city
0,0,MOBILE VENDING (FOOD TRUCK),R54-4422459,NaN,2014-05-16,"AHMADZADEH-RAZAVI, MAHNAZ",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-08-16,2015-08-16,"canada,l6p",43.794049,-79.702103,"L6P, Brampton, Ontario","L6P, Brampton, Ontario",Brampton
1,1,MOBILE VENDING (FOOD TRUCK),R54-4422376,NaN,2014-05-15,TANG'S FAMILY RESTAURANT CORP,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-09-10,2015-09-10,"3380 midland ave, #19,toronto, on,canada,m1v 5b5",43.809303,-79.290346,"3380 Midland Ave, Unit 19, Scarborough, Ontari...","3380 Midland Ave, Unit 19, Scarborough, Ontari...",Scarborough
2,2,MOBILE VENDING (FOOD TRUCK),R54-4424708,NaN,2014-07-09,"INSRICHIENGMAI, SUSAN",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-10-08,2015-10-08,"canada,m6k",43.637944,-79.429962,"M6K, Toronto, Ontario","M6K, Toronto, Ontario",Toronto
3,3,MOBILE VENDING (FOOD TRUCK),R54-4422377,NaN,2014-05-15,1459949 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"90 esna park dr, #22,markham, on,canada,l3r 2r7",43.827610,-79.338622,"90 Esna Park Dr, Unit 22, Markham, Ontario, L3...","90 Esna Park Dr, Unit 22, Markham, Ontario, L3...",Markham
4,4,MOBILE VENDING (FOOD TRUCK),R54-4422545,NaN,2014-05-20,2397349 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"1621 skyline dr,mississauga, on,canada,l5e 2w9",43.599759,-79.558298,"1621 Skyline Dr, Mississauga, Ontario, L5E 2W9","1621 Skyline Dr, Mississauga, Ontario, L5E 2W9",Mississauga


In [5]:
df.shape

(3332, 18)

# OpenAI Functions

## Initializing completion function and parser

In [6]:
# Open the JSON file
with open('/Users/edwin/Repositories/api_keys.json') as file:
    data = json.load(file)

# Access the API keys
openai.api_key= data['apiKeys']['OpenAI2']

In [7]:
def get_completion(prompt, model="gpt-4",temperature=0): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
def output_parser(response):
    response=response.split(',')
    list_oflists=[]
    for each in response: 
        list_todf=each.split("-")
        
        if len(list_todf)==3:
            list_oflists.append(list_todf)
        else: pass
    return pd.DataFrame(list_oflists,columns=["BusinessNo","Category","Subcategory"])

## Prompt and iterative function

In [9]:
def prompt_formatter(df_toparse):

    length=str(df_toparse.shape[0])

    base_prompt= f"""
        Given the information about businesses, infer the appropriate category and subcategory 
        for each business primarily based on its Operating Name. If the Operating Name does not 
        provide a clear indication, proceed to analyze the Client Name. If it is still unclear, 
        then use the Conditions or the Free Form Conditions for guidance. 
        If the information provided does not allow for a clear categorization, 
        mark the category or subcategory as 'Unknown'.

        The output instructions are inside the following triple backticks:
        '''
        The output SHOULD be formatted by Business Number, Category and subcategory, and separated with commas as inside the triple backticks: 
        '''Business Number - Category - Subcategory, Business Number - Category - Subcategory'''

        For example: 
        1 - Asian Cuisine - Japanese, 2- American Cuisine - Burgers and Sandwiches, ...
        
        Ensure that each business has those 3 mentioned fields, and that are in the same order(Business number - Category - Subcategory).

        Do not add any additional information in the output or any of the tag names like "Category" or "Subcategories" or the name of the business.
        '''
  

        The categories and subcategories are inside the following triple backticks:
        '''
        - American Cuisine
            - Burgers and Sandwiches
            - Barbecue
            - Fried Foods
            - Other American Food

        - Canadian Cuisine
            - Poutine
            - Canadian Bacon or Peameal Bacon
            - Butter Tarts
            - Maple Syrup Products
            - Other Canadian Food

        - Mexican Cuisine
            - Tacos and Burritos
            - Street Corn and Snacks
            - Other Mexican Food

        - Asian Cuisine
            - Chinese
            - Japanese
            - Korean
            - Southeast Asian
            - Other Asian Food

        - Mediterranean & Middle Eastern
            - Gyros and Shawarma
            - Greek
            - Middle Eastern
            - Other Mediterranean Food

        - Indian Cuisine
            - Curries and Tandoori
            - Other Indian Food

        - Italian Cuisine
            - Pizza and Pasta
            - Italian Street Food
            - Other Italian Food

        - Desserts and Sweets
            - Ice Cream and Frozen Treats
            - Baked Goods and Pastries
            - Crepes and Waffles
            - Other Desserts

        - Beverages
            - Coffee and Tea
            - Juice Bars
            - Other Beverages

        - Other international food
            - African Cuisine
            - South American Cuisine
            - Central American Cuisine
            - Caribbean Cuisine
            - European Cuisine
            - Oceanian Cuisine
            - Other Global Specialties

        - Specialty and Fusion
            - Health and Wellness
            - Fusion
            - Gourmet and Specialty
            - Other Specialty Food
        '''

        The {length} businesses information are inside the following triple backticks separated by commas:
        '''
    """
    formatted_strings = []

    for index, row in df_toparse.iterrows():
        formatted_string = f"""
        Business Number {row['BusinessNoOther']}:[
            Operating Name: {row['Operating Name']},
            Client Name: {row['Client Name']},
            Conditions: {row['Conditions']},
            Free Form Conditions Line 1: {row['Free Form Conditions Line 1']}
            Free Form Conditions Line 2: {row['Free Form Conditions Line 2']}
            ],
            """
        formatted_strings.append(formatted_string)
    
    # Joining all formatted strings to display
    formatted_output = "\n".join(formatted_strings)

    return base_prompt+formatted_output+"'''"


# Iterating

In [10]:
df_other=pd.read_csv("df_other.csv").drop(columns="Unnamed: 0")

In [11]:
df_other=df_other.reset_index().rename(columns={"index":"BusinessNoOther"})
df_other.head()

,BusinessNoOther,BusinessNo,Category,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city
0,0,0,MOBILE VENDING (FOOD TRUCK),R54-4422459,NaN,2014-05-16,"AHMADZADEH-RAZAVI, MAHNAZ",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-08-16,2015-08-16,"canada,l6p",43.794049,-79.702103,"L6P, Brampton, Ontario","L6P, Brampton, Ontario",Brampton
1,1,1,MOBILE VENDING (FOOD TRUCK),R54-4422376,NaN,2014-05-15,TANG'S FAMILY RESTAURANT CORP,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-09-10,2015-09-10,"3380 midland ave, #19,toronto, on,canada,m1v 5b5",43.809303,-79.290346,"3380 Midland Ave, Unit 19, Scarborough, Ontari...","3380 Midland Ave, Unit 19, Scarborough, Ontari...",Scarborough
2,2,2,MOBILE VENDING (FOOD TRUCK),R54-4424708,NaN,2014-07-09,"INSRICHIENGMAI, SUSAN",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-10-08,2015-10-08,"canada,m6k",43.637944,-79.429962,"M6K, Toronto, Ontario","M6K, Toronto, Ontario",Toronto
3,3,3,MOBILE VENDING (FOOD TRUCK),R54-4422377,NaN,2014-05-15,1459949 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"90 esna park dr, #22,markham, on,canada,l3r 2r7",43.827610,-79.338622,"90 Esna Park Dr, Unit 22, Markham, Ontario, L3...","90 Esna Park Dr, Unit 22, Markham, Ontario, L3...",Markham
4,4,4,MOBILE VENDING (FOOD TRUCK),R54-4422545,NaN,2014-05-20,2397349 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"1621 skyline dr,mississauga, on,canada,l5e 2w9",43.599759,-79.558298,"1621 Skyline Dr, Mississauga, Ontario, L5E 2W9","1621 Skyline Dr, Mississauga, Ontario, L5E 2W9",Mississauga


In [12]:
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(df_other), 30))
index_list

[0,
 30,
 60,
 90,
 120,
 150,
 180,
 210,
 240,
 270,
 300,
 330,
 360,
 390,
 420,
 450,
 480,
 510,
 540,
 570,
 600,
 630,
 660,
 690,
 720,
 750,
 780,
 810,
 840,
 870,
 900,
 930,
 960,
 990,
 1020,
 1050,
 1080,
 1110,
 1140,
 1170,
 1200,
 1230,
 1260,
 1290,
 1320,
 1350,
 1380,
 1410,
 1440,
 1470,
 1500,
 1530,
 1560,
 1590,
 1620,
 1650,
 1680,
 1710,
 1740,
 1770,
 1800,
 1830,
 1860,
 1890,
 1920,
 1950,
 1980,
 2010,
 2040,
 2070,
 2100,
 2130,
 2160,
 2190,
 2220,
 2250,
 2280,
 2310,
 2340,
 2370,
 2400,
 2430,
 2460,
 2490,
 2520,
 2550,
 2580,
 2610,
 2640,
 2670,
 2700,
 2730,
 2760,
 2790,
 2820,
 2850,
 2853]

In [13]:
last_index=0

In [14]:
df_iterated=pd.DataFrame([],columns=["BusinessNo","Category","Subcategory"])

In [15]:
# Loop through the index_list
for i in range(last_index, len(index_list)-1):
    df_i = df_other[index_list[i]:index_list[i+1]]
    prompt=prompt_formatter(df_i)
    try:
        response = get_completion(prompt, temperature=0)
        parsed_response=output_parser(response)
        df_iterated=pd.concat([df_iterated,parsed_response])
        last_index=i
        ic(last_index,index_list[i],index_list[i+1],'successful',len(df_iterated))
    except: 
        last_index=i
        print(last_index)
        break

ic| last_index: 0
    index_list[i]: 0
    index_list[i+1]: 30
    'successful': 'successful'
    len(df_iterated): 30
ic| last_index: 1
    index_list[i]: 30
    index_list[i+1]: 60
    'successful': 'successful'
    len(df_iterated): 60
ic| last_index: 2
    index_list[i]: 60
    index_list[i+1]: 90
    'successful': 'successful'
    len(df_iterated): 90
ic| last_index: 3
    index_list[i]: 90
    index_list[i+1]: 120
    'successful': 'successful'
    len(df_iterated): 120
ic| last_index: 4
    index_list[i]: 120
    index_list[i+1]: 150
    'successful': 'successful'
    len(df_iterated): 150
ic| last_index: 5
    index_list[i]: 150
    index_list[i+1]: 180
    'successful': 'successful'
    len(df_iterated): 180
ic| last_index: 6
    index_list[i]: 180
    index_list[i+1]: 210
    'successful': 'successful'
    len(df_iterated): 210
ic| last_index: 7
    index_list[i]: 210
    index_list[i+1]: 240
    'successful': 'successful'
    len(df_iterated): 240
ic| last_index: 8
    index

In [17]:
df_iterated.to_csv('businesscategories_tocsv.csv')
df_iterated

,BusinessNo,Category,Subcategory
0,0,Unknown,Unknown
1,1,Asian Cuisine,Other Asian Food
2,2,Unknown,Unknown
3,3,Unknown,Unknown
4,4,Unknown,Unknown
...,...,...,...
28,2848,Desserts and Sweets,Ice Cream and Frozen Treats
29,2849,Mexican Cuisine,Other Mexican Food
0,2850,American Cuisine,Burgers and Sandwiches
1,2851,American Cuisine,Burgers and Sandwiches


In [23]:
df_iterated["BusinessNo"]=df_iterated.BusinessNo.str.strip()
df_iterated["Category"]=df_iterated.Category.str.strip()
df_iterated["Subcategory"]=df_iterated.Subcategory.str.strip()

In [25]:
df_iterated=df_iterated.reset_index().drop(columns="index")
df_iterated.head()

,BusinessNo,Category,Subcategory
0,0,Unknown,Unknown
1,1,Asian Cuisine,Other Asian Food
2,2,Unknown,Unknown
3,3,Unknown,Unknown
4,4,Unknown,Unknown


In [27]:
df_iterated['BusinessNo'] = pd.to_numeric(df_iterated['BusinessNo'], errors='coerce')
df_iterated

,BusinessNo,Category,Subcategory
0,0.0,Unknown,Unknown
1,1.0,Asian Cuisine,Other Asian Food
2,2.0,Unknown,Unknown
3,3.0,Unknown,Unknown
4,4.0,Unknown,Unknown
...,...,...,...
2848,2848.0,Desserts and Sweets,Ice Cream and Frozen Treats
2849,2849.0,Mexican Cuisine,Other Mexican Food
2850,2850.0,American Cuisine,Burgers and Sandwiches
2851,2851.0,American Cuisine,Burgers and Sandwiches


## Merging 2 datasets

In [29]:
final_df_wcategories=df_other.merge(df_iterated,how='left',left_on='BusinessNoOther',right_on='BusinessNo').drop(columns={"BusinessNoOther","BusinessNo_y"}).rename(columns={"BusinessNo_x":"BusinessNo","Category_x":"Category_Vendor","Category_y":"Category"})
final_df_wcategories

,BusinessNo,Category_Vendor,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city,Category,Subcategory
0,0,MOBILE VENDING (FOOD TRUCK),R54-4422459,NaN,2014-05-16,"AHMADZADEH-RAZAVI, MAHNAZ",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-08-16,2015-08-16,"canada,l6p",43.794049,-79.702103,"L6P, Brampton, Ontario","L6P, Brampton, Ontario",Brampton,Unknown,Unknown
1,1,MOBILE VENDING (FOOD TRUCK),R54-4422376,NaN,2014-05-15,TANG'S FAMILY RESTAURANT CORP,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-09-10,2015-09-10,"3380 midland ave, #19,toronto, on,canada,m1v 5b5",43.809303,-79.290346,"3380 Midland Ave, Unit 19, Scarborough, Ontari...","3380 Midland Ave, Unit 19, Scarborough, Ontari...",Scarborough,Asian Cuisine,Other Asian Food
2,2,MOBILE VENDING (FOOD TRUCK),R54-4424708,NaN,2014-07-09,"INSRICHIENGMAI, SUSAN",NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2015-10-08,2015-10-08,"canada,m6k",43.637944,-79.429962,"M6K, Toronto, Ontario","M6K, Toronto, Ontario",Toronto,Unknown,Unknown
3,3,MOBILE VENDING (FOOD TRUCK),R54-4422377,NaN,2014-05-15,1459949 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"90 esna park dr, #22,markham, on,canada,l3r 2r7",43.827610,-79.338622,"90 Esna Park Dr, Unit 22, Markham, Ontario, L3...","90 Esna Park Dr, Unit 22, Markham, Ontario, L3...",Markham,Unknown,Unknown
4,4,MOBILE VENDING (FOOD TRUCK),R54-4422545,NaN,2014-05-20,2397349 ONTARIO INC,NaN,NaN,NaN,MOBILE VENDING (FOOD TRUCK) ANNUAL;,2016-01-06,2016-01-06,"1621 skyline dr,mississauga, on,canada,l5e 2w9",43.599759,-79.558298,"1621 Skyline Dr, Mississauga, Ontario, L5E 2W9","1621 Skyline Dr, Mississauga, Ontario, L5E 2W9",Mississauga,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,3327,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,V27-5338382,GOOD BEHAVIOUR ICE CREAM,2023-07-20,GOOD BEHAVIOUR ICE CREAM INC,MUST COMPLY WITH ALL CITY AND ZONING BY-LAWS;P...,NaN,NaN,RV - NON-MOTORIZED - ICE CREAM;,NaN,2023-07-20,"393 woodfield rd,toronto, on,canada,m4l 2x2",43.674600,-79.324858,"393 Woodfield Rd, Toronto, Ontario, M4L 2X2","393 Woodfield Rd, Toronto, Ontario, M4L 2X2",Toronto,Desserts and Sweets,Ice Cream and Frozen Treats
2849,3328,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,V27-5345989,CHURROS BY REBOZOS,2023-08-04,CHURROS BY REBOZOS INC,MUST COMPLY WITH ALL CITY AND ZONING BY-LAWS;C...,ALSO SELLING TACOS AND QUESADILLAS,NaN,RV - NON-MOTORIZED - PROPANE HEATER;,NaN,2023-10-27,"126 rogers rd,toronto, on,canada,m6e 1p7",43.684813,-79.449630,"126 Rogers Rd, York, Ontario, M6E 1P7","126 Rogers Rd, York, Ontario, M6E 1P7",York,Mexican Cuisine,Other Mexican Food
2850,3329,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,V27-5349014,FAST FOOD CART,2023-09-15,"BASHARDOST, FOLURA",MUST COMPLY WITH ALL CITY AND ZONING BY-LAWS;C...,NaN,NaN,RV - NON-MOTORIZED - PROPANE HEATER;,NaN,2023-09-15,"canada,m1s",43.794119,-79.265591,"M1S, Toronto, Ontario","M1S, Toronto, Ontario",Toronto,American Cuisine,Burgers and Sandwiches
2851,3330,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,V27-5377789,BEE BOX FOODS,2023-12-18,"BALDEO, PETER TONY",MUST COMPLY WITH ALL CITY AND ZONING BY-LAWS;C...,NaN,NaN,RV - TRAILER - ELECTRIC;,NaN,2023-12-18,"canada,m2n",43.767597,-79.410276,"M2N, Toronto, Ontario","M2N, Toronto, Ontario",Toronto,American Cuisine,Burgers and Sandwiches


In [39]:
df_icecream=pd.read_csv("df_icecream.csv").drop(columns="Unnamed: 0").rename(columns={"Category":"Category_Vendor"})
df_icecream.head()

,BusinessNo,Category_Vendor,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city
0,280,MOBILE VENDING (ICE CREAM TRUCK),R56-4536547,NaN,2015-05-14,1544562 ONTARIO LTD,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) 6-MONTH;,2015-05-15,2017-09-08,"80 oakwood ave,toronto, on,canada,m6h 2v8",43.677013,-79.435124,"80 Oakwood Ave, Toronto, Ontario, M6H 2V8","80 Oakwood Ave, Toronto, Ontario, M6H 2V8",Toronto
1,281,MOBILE VENDING (ICE CREAM TRUCK),R56-4422772,NaN,2014-05-26,"STAVRESKI, NAUMCE",NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-07-02,2015-07-02,"canada,m1k",43.729683,-79.263810,"M1K, Scarborough, Ontario","M1K, Scarborough, Ontario",Scarborough
2,282,MOBILE VENDING (ICE CREAM TRUCK),R56-4422408,NaN,2014-05-15,MASTER SOFT & DELIGHT DAIRY PRODUCTS INC,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-07-13,2017-09-08,"58 raleigh ave,toronto, on,canada,m1k 1a3",43.704275,-79.264846,"58 Raleigh Ave, Scarborough, Ontario, M1K 1A3","58 Raleigh Ave, Scarborough, Ontario, M1K 1A3",Scarborough
3,283,MOBILE VENDING (ICE CREAM TRUCK),R56-4427339,NaN,2014-09-05,1908280 ONTARIO LTD,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-08-10,2017-09-08,"58 pinewood ave,toronto, on,canada,m6c 2v1",43.684392,-79.425782,"58 Pinewood Ave, York, Ontario, M6C 2V1","58 Pinewood Ave, York, Ontario, M6C 2V1",York
4,284,MOBILE VENDING (ICE CREAM TRUCK),R56-4422385,NaN,2014-05-15,"SOHRABI, ARZHANG",NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-08-21,2015-08-21,"canada,l4c",43.867765,-79.440645,"L4C, Richmond Hill, Ontario","L4C, Richmond Hill, Ontario",Richmond Hill


In [40]:
df_icecream["Category"]="Desserts and Sweets"
df_icecream["Subcategory"]="Ice Cream and Frozen Treats"
df_icecream.head()

,BusinessNo,Category_Vendor,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city,Category,Subcategory
0,280,MOBILE VENDING (ICE CREAM TRUCK),R56-4536547,NaN,2015-05-14,1544562 ONTARIO LTD,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) 6-MONTH;,2015-05-15,2017-09-08,"80 oakwood ave,toronto, on,canada,m6h 2v8",43.677013,-79.435124,"80 Oakwood Ave, Toronto, Ontario, M6H 2V8","80 Oakwood Ave, Toronto, Ontario, M6H 2V8",Toronto,Desserts and Sweets,Ice Cream and Frozen Treats
1,281,MOBILE VENDING (ICE CREAM TRUCK),R56-4422772,NaN,2014-05-26,"STAVRESKI, NAUMCE",NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-07-02,2015-07-02,"canada,m1k",43.729683,-79.263810,"M1K, Scarborough, Ontario","M1K, Scarborough, Ontario",Scarborough,Desserts and Sweets,Ice Cream and Frozen Treats
2,282,MOBILE VENDING (ICE CREAM TRUCK),R56-4422408,NaN,2014-05-15,MASTER SOFT & DELIGHT DAIRY PRODUCTS INC,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-07-13,2017-09-08,"58 raleigh ave,toronto, on,canada,m1k 1a3",43.704275,-79.264846,"58 Raleigh Ave, Scarborough, Ontario, M1K 1A3","58 Raleigh Ave, Scarborough, Ontario, M1K 1A3",Scarborough,Desserts and Sweets,Ice Cream and Frozen Treats
3,283,MOBILE VENDING (ICE CREAM TRUCK),R56-4427339,NaN,2014-09-05,1908280 ONTARIO LTD,NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-08-10,2017-09-08,"58 pinewood ave,toronto, on,canada,m6c 2v1",43.684392,-79.425782,"58 Pinewood Ave, York, Ontario, M6C 2V1","58 Pinewood Ave, York, Ontario, M6C 2V1",York,Desserts and Sweets,Ice Cream and Frozen Treats
4,284,MOBILE VENDING (ICE CREAM TRUCK),R56-4422385,NaN,2014-05-15,"SOHRABI, ARZHANG",NaN,NaN,NaN,MOBILE VENDING (ICE CREAM TRUCK) ANNUAL;,2015-08-21,2015-08-21,"canada,l4c",43.867765,-79.440645,"L4C, Richmond Hill, Ontario","L4C, Richmond Hill, Ontario",Richmond Hill,Desserts and Sweets,Ice Cream and Frozen Treats


In [44]:
final_df_wcategories=pd.concat([final_df_wcategories,df_icecream])

In [45]:
final_df_wcategories["Category_Vendor"]=final_df_wcategories.Category_Vendor.str.lower()
final_df_wcategories["Operating Name"]=final_df_wcategories["Operating Name"].str.title()
final_df_wcategories["Client Name"]=final_df_wcategories["Client Name"].str.title()
final_df_wcategories["Endorsements"]=final_df_wcategories["Endorsements"].str.lower()
final_df_wcategories

,BusinessNo,Category_Vendor,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city,Category,Subcategory
0,0,mobile vending (food truck),R54-4422459,NaN,2014-05-16,"Ahmadzadeh-Razavi, Mahnaz",NaN,NaN,NaN,mobile vending (food truck) annual;,2015-08-16,2015-08-16,"canada,l6p",43.794049,-79.702103,"L6P, Brampton, Ontario","L6P, Brampton, Ontario",Brampton,Unknown,Unknown
1,1,mobile vending (food truck),R54-4422376,NaN,2014-05-15,Tang'S Family Restaurant Corp,NaN,NaN,NaN,mobile vending (food truck) annual;,2015-09-10,2015-09-10,"3380 midland ave, #19,toronto, on,canada,m1v 5b5",43.809303,-79.290346,"3380 Midland Ave, Unit 19, Scarborough, Ontari...","3380 Midland Ave, Unit 19, Scarborough, Ontari...",Scarborough,Asian Cuisine,Other Asian Food
2,2,mobile vending (food truck),R54-4424708,NaN,2014-07-09,"Insrichiengmai, Susan",NaN,NaN,NaN,mobile vending (food truck) annual;,2015-10-08,2015-10-08,"canada,m6k",43.637944,-79.429962,"M6K, Toronto, Ontario","M6K, Toronto, Ontario",Toronto,Unknown,Unknown
3,3,mobile vending (food truck),R54-4422377,NaN,2014-05-15,1459949 Ontario Inc,NaN,NaN,NaN,mobile vending (food truck) annual;,2016-01-06,2016-01-06,"90 esna park dr, #22,markham, on,canada,l3r 2r7",43.827610,-79.338622,"90 Esna Park Dr, Unit 22, Markham, Ontario, L3...","90 Esna Park Dr, Unit 22, Markham, Ontario, L3...",Markham,Unknown,Unknown
4,4,mobile vending (food truck),R54-4422545,NaN,2014-05-20,2397349 Ontario Inc,NaN,NaN,NaN,mobile vending (food truck) annual;,2016-01-06,2016-01-06,"1621 skyline dr,mississauga, on,canada,l5e 2w9",43.599759,-79.558298,"1621 Skyline Dr, Mississauga, Ontario, L5E 2W9","1621 Skyline Dr, Mississauga, Ontario, L5E 2W9",Mississauga,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,754,mobile vending (ice cream truck),R56-5318650,Sweet Summer Soft Ice,2023-05-01,1532272 Ontario Corp,NaN,NaN,NaN,mobile vending (ice cream truck) annual;,NaN,2023-05-05,"89 pamille pl,toronto, on,canada,m6m 3b1",43.700999,-79.499614,"89 Pamille Pl, North York, Ontario, M6M 3B1","89 Pamille Pl, North York, Ontario, M6M 3B1",North York,Desserts and Sweets,Ice Cream and Frozen Treats
475,755,mobile vending (ice cream truck),R56-5318652,Sweet Summer Soft Icecream,2023-05-01,1532272 Ontario Corp,NaN,NaN,NaN,mobile vending (ice cream truck) annual;,NaN,2023-05-05,"89 pamille pl,toronto, on,canada,m6m 3b1",43.700999,-79.499614,"89 Pamille Pl, North York, Ontario, M6M 3B1","89 Pamille Pl, North York, Ontario, M6M 3B1",North York,Desserts and Sweets,Ice Cream and Frozen Treats
476,756,mobile vending (ice cream truck),R56-5342113,Mrs Softy,2023-07-26,Mrs Softy Inc,NaN,NaN,NaN,mobile vending (ice cream truck) 6-month;,NaN,2023-11-24,"144 clifton ave,toronto, on,canada,m3h 4l5",43.759783,-79.459992,"144 Clifton Ave, North York, Ontario, M3H 4L5","144 Clifton Ave, North York, Ontario, M3H 4L5",North York,Desserts and Sweets,Ice Cream and Frozen Treats
477,757,mobile vending (ice cream truck),R56-5343744,Master Soft & Delight Dairy Products,2023-07-26,Master Soft & Delight Dairy Products Inc,NaN,NaN,NaN,mobile vending (ice cream truck) 6-month;,NaN,2024-01-03,"58 raleigh ave,toronto, on,canada,m1k 1a3",43.704275,-79.264846,"58 Raleigh Ave, Scarborough, Ontario, M1K 1A3","58 Raleigh Ave, Scarborough, Ontario, M1K 1A3",Scarborough,Desserts and Sweets,Ice Cream and Frozen Treats


In [46]:
final_df_wcategories["Conditions"]=final_df_wcategories.Conditions.str.lower()
final_df_wcategories["Free Form Conditions Line 1"]=final_df_wcategories["Free Form Conditions Line 1"].str.lower()
final_df_wcategories["Free Form Conditions Line 2"]=final_df_wcategories["Free Form Conditions Line 2"].str.lower()
final_df_wcategories

,BusinessNo,Category_Vendor,Licence No.,Operating Name,Issued,Client Name,Conditions,Free Form Conditions Line 1,Free Form Conditions Line 2,Endorsements,Cancel Date,Last Record Update,address,latitude,longitude,formatted_address,location,city,Category,Subcategory
0,0,mobile vending (food truck),R54-4422459,NaN,2014-05-16,"Ahmadzadeh-Razavi, Mahnaz",NaN,NaN,NaN,mobile vending (food truck) annual;,2015-08-16,2015-08-16,"canada,l6p",43.794049,-79.702103,"L6P, Brampton, Ontario","L6P, Brampton, Ontario",Brampton,Unknown,Unknown
1,1,mobile vending (food truck),R54-4422376,NaN,2014-05-15,Tang'S Family Restaurant Corp,NaN,NaN,NaN,mobile vending (food truck) annual;,2015-09-10,2015-09-10,"3380 midland ave, #19,toronto, on,canada,m1v 5b5",43.809303,-79.290346,"3380 Midland Ave, Unit 19, Scarborough, Ontari...","3380 Midland Ave, Unit 19, Scarborough, Ontari...",Scarborough,Asian Cuisine,Other Asian Food
2,2,mobile vending (food truck),R54-4424708,NaN,2014-07-09,"Insrichiengmai, Susan",NaN,NaN,NaN,mobile vending (food truck) annual;,2015-10-08,2015-10-08,"canada,m6k",43.637944,-79.429962,"M6K, Toronto, Ontario","M6K, Toronto, Ontario",Toronto,Unknown,Unknown
3,3,mobile vending (food truck),R54-4422377,NaN,2014-05-15,1459949 Ontario Inc,NaN,NaN,NaN,mobile vending (food truck) annual;,2016-01-06,2016-01-06,"90 esna park dr, #22,markham, on,canada,l3r 2r7",43.827610,-79.338622,"90 Esna Park Dr, Unit 22, Markham, Ontario, L3...","90 Esna Park Dr, Unit 22, Markham, Ontario, L3...",Markham,Unknown,Unknown
4,4,mobile vending (food truck),R54-4422545,NaN,2014-05-20,2397349 Ontario Inc,NaN,NaN,NaN,mobile vending (food truck) annual;,2016-01-06,2016-01-06,"1621 skyline dr,mississauga, on,canada,l5e 2w9",43.599759,-79.558298,"1621 Skyline Dr, Mississauga, Ontario, L5E 2W9","1621 Skyline Dr, Mississauga, Ontario, L5E 2W9",Mississauga,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,754,mobile vending (ice cream truck),R56-5318650,Sweet Summer Soft Ice,2023-05-01,1532272 Ontario Corp,NaN,NaN,NaN,mobile vending (ice cream truck) annual;,NaN,2023-05-05,"89 pamille pl,toronto, on,canada,m6m 3b1",43.700999,-79.499614,"89 Pamille Pl, North York, Ontario, M6M 3B1","89 Pamille Pl, North York, Ontario, M6M 3B1",North York,Desserts and Sweets,Ice Cream and Frozen Treats
475,755,mobile vending (ice cream truck),R56-5318652,Sweet Summer Soft Icecream,2023-05-01,1532272 Ontario Corp,NaN,NaN,NaN,mobile vending (ice cream truck) annual;,NaN,2023-05-05,"89 pamille pl,toronto, on,canada,m6m 3b1",43.700999,-79.499614,"89 Pamille Pl, North York, Ontario, M6M 3B1","89 Pamille Pl, North York, Ontario, M6M 3B1",North York,Desserts and Sweets,Ice Cream and Frozen Treats
476,756,mobile vending (ice cream truck),R56-5342113,Mrs Softy,2023-07-26,Mrs Softy Inc,NaN,NaN,NaN,mobile vending (ice cream truck) 6-month;,NaN,2023-11-24,"144 clifton ave,toronto, on,canada,m3h 4l5",43.759783,-79.459992,"144 Clifton Ave, North York, Ontario, M3H 4L5","144 Clifton Ave, North York, Ontario, M3H 4L5",North York,Desserts and Sweets,Ice Cream and Frozen Treats
477,757,mobile vending (ice cream truck),R56-5343744,Master Soft & Delight Dairy Products,2023-07-26,Master Soft & Delight Dairy Products Inc,NaN,NaN,NaN,mobile vending (ice cream truck) 6-month;,NaN,2024-01-03,"58 raleigh ave,toronto, on,canada,m1k 1a3",43.704275,-79.264846,"58 Raleigh Ave, Scarborough, Ontario, M1K 1A3","58 Raleigh Ave, Scarborough, Ontario, M1K 1A3",Scarborough,Desserts and Sweets,Ice Cream and Frozen Treats


In [47]:
final_df_wcategories.groupby(by=["Category","Subcategory"]).count().BusinessNo

Category                        Subcategory                    
African Cuisine                 Other Global Specialties             2
American Cuisine                Barbecue                            18
                                Burgers and Sandwiches             818
                                Fried Foods                         16
                                Hot Dogs                            87
                                Other American Food                232
                                Other American Food"                 1
                                Sandwiches                           2
Asian Cuisine                   Chinese                             15
                                Japanese                             2
                                Korean                               6
                                Other Asian Food                    25
                                Southeast Asian                     10
Beverages    

In [48]:
final_df_wcategories.to_csv("final_df_wcategories.csv")